<a href="https://colab.research.google.com/github/mertgulgel/sum/blob/main/sum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber
!pip install PyMuPDF
!pip install os
!pip install PyPDF2
!pip install re
!pip install transformers
!pip install pdfminer


In [ ]:
!pip install csv
!pip install requests
!pip install bs4

In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
import re

linkler = []

with open('veriler.csv', mode='r', newline='', encoding='utf-8') as dosya:
    okuyucu = csv.reader(dosya)
    for satir in okuyucu:
        link = satir[1]
        linkler.append(link)

for i in range(50):
    url = linkler[i+1]
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    print(url)

    download_link = []
    pdf_link = soup.find('meta', {'name': 'citation_pdf_url'})
    if pdf_link is not None:
        pdf_url = pdf_link['content']

        response = requests.get(pdf_url)
        dergi_adi = pdf_url.split('/')[-1]
        with open(f'pdfler/{dergi_adi}.pdf', 'wb') as dosya:
            dosya.write(response.content)
        print(f'{dergi_adi}.pdf indirildi')
    else:
        print("URL Not Found")

In [11]:
!pip install pdfplumber==0.11.0

  Using cached pdfplumber-0.11.0-py3-none-any.whl (56 kB)
  Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20200517
    Uninstalling pdfminer.six-20200517:
      Successfully uninstalled pdfminer.six-20200517
  Attempting uninstall: pdfplumber
    Found existing installation: pdfplumber 0.5.25
    Uninstalling pdfplumber-0.5.25:
      Successfully uninstalled pdfplumber-0.5.25


In [8]:
import os
import pdfplumber
import fitz  # PyMuPDF
from pdfminer.pdfparser import PDFSyntaxError
import PyPDF2
import re

from transformers import pipeline, AutoTokenizer, TFAutoModelForSeq2SeqLM

# Daha küçük bir model seçmek için model ve tokenizer'ı değiştirin
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

pdf_folder_path = r'pdfler'
text_folder_path = r'textler'

pdf_files = [f for f in os.listdir(pdf_folder_path) if f.endswith('.pdf')]
text_files = [s for s in os.listdir(text_folder_path) if s.endswith('.txt')]

summarization_pipeline = pipeline("summarization", model=model, tokenizer=tokenizer)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [2]:
def extract_text_from_pdf(file_path):
    text = ""
    try:
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"

    except PDFSyntaxError as e:
        print(f"Syntax error reading {file_path}: {e}")
        # Attempt text extraction using PyMuPDF as an alternative method
        try:
            doc = fitz.open(file_path)
            for page_number in range(len(doc)):
                page = doc.load_page(page_number)
                text += page.get_text("text")
        except Exception as e:
            print(f"Error extracting text using PyMuPDF: {e}")
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        # Handle other exceptions here
    return text


In [3]:
for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_folder_path, pdf_file)

    try:
        pdf_text = extract_text_from_pdf(pdf_path)
        if len(pdf_text) != 0:
            with open(f'textler/{pdf_file}.txt', 'w', encoding='utf-8') as dosya:
                dosya.write(pdf_text)
    except PDFSyntaxError as e:
        print(f"Syntax error reading {pdf_file}: {e}")
        print(f"Skipping summarization for {pdf_file}.")
        continue

    if not pdf_text:
        print(f"No text extracted from {pdf_file}. Skipping summarization.")
        continue

No text extracted from 1172021.pdf. Skipping summarization.
Syntax error reading pdfler/764679.pdf: No /Root object! - Is this really a PDF?
Error extracting text using PyMuPDF: Failed to open file 'pdfler/764679.pdf'.
No text extracted from 764679.pdf. Skipping summarization.
No text extracted from 116579.pdf. Skipping summarization.
No text extracted from 1097795.pdf. Skipping summarization.
No text extracted from 38227.pdf. Skipping summarization.
No text extracted from 945918.pdf. Skipping summarization.
No text extracted from 436797.pdf. Skipping summarization.
No text extracted from 321669.pdf. Skipping summarization.
No text extracted from 1775985.pdf. Skipping summarization.
No text extracted from 788938.pdf. Skipping summarization.


In [ ]:
def extract_text_from_txt(txt_path):
    with open(txt_path, 'r', encoding='utf-8') as file:
        return file.read()

max_input_length = 512  # Max input length for the model

for text_file in text_files:
    text_path = os.path.join(text_folder_path, text_file)
    print(text_path)
    try:
        text_text = extract_text_from_txt(text_path)
        def preprocess_text(text):
            text = text.lower()
            text = re.sub(r'[^\w\s]', '', text)
            text = re.sub(r'\d+|\s+', '', text)
            return text
        processed_text = preprocess_text(text_text)
        segments = [processed_text[i:i+max_input_length] for i in range(0, len(processed_text), max_input_length)]
        summarized_segments = []
        for segment in segments:
            summarized_segment = summarization_pipeline(segment, max_length=49, min_length=30)[0]['summary_text']
            summarized_segments.append(summarized_segment)
        summarized_text = ' '.join(summarized_segments)
        with open(f'summariez/{text_file}.txt', 'w', encoding='utf-8') as file:
            file.write(summarized_text)
    except Exception as e:
        print(f"Error reading {text_file}: {e}")
        print(f"Skipping summarization for {text_file}.")
        continue

    if not text_text:
        print(f"No text extracted from {text_file}. Skipping summarization.")
        continue

textler/2019455.pdf.txt
